In [32]:
import pandas as pd
__file__=''
import os,sys
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../code/LIB/'))
from env import FILE
import pickle

In [80]:
tags = 'new_model_RNN_OPTIMIZE_Utag_NOTNA.csv'
notags = 'new_model_RNN_OPTIMIZE_Utag_NA.csv'
df_tags = pd.read_csv('../test/{}'.format(tags))
df_notags = pd.read_csv('../test/{}'.format(notags))


test_ori = pd.read_pickle(FILE.test_ori.value)
print(df_notags.shape)

df_save_tag = test_ori[test_ori['user_tags'].notnull()][['instance_id']].copy()
df_save_notag = test_ori[test_ori['user_tags'].isnull()][['instance_id']].copy()

df_save_tag = df_save_tag.merge(df_tags,how='inner',on='instance_id')
print(df_save_tag.shape)
df_save_notag = df_save_notag.merge(df_notags,how='inner',on='instance_id')
print(df_save_notag.shape)
df_save = test_ori[['instance_id']].copy()
df_save = df_save.merge(pd.concat([df_save_tag,df_save_notag],sort=False),how='inner',on='instance_id')
print(df_save.shape)

(40024, 2)
(26792, 2)
(13232, 2)
(40024, 2)


# TRAIN MERGE

In [77]:
tags = 'new_model_RNN_OPTIMIZE_Utag_NOTNA.pkl'
notags = 'new_model_RNN_OPTIMIZE_Utag_NA.pkl'
df_tags = pd.read_pickle('../train/{}'.format(tags))
df_notags = pd.read_pickle('../train/{}'.format(notags))


train_ori = pd.read_pickle(FILE.train_ori.value)
print(df_notags.shape)

df_save_tag = train_ori[train_ori['user_tags'].notnull()][['instance_id']].copy()
df_save_notag = train_ori[train_ori['user_tags'].isnull()][['instance_id']].copy()

df_save_tag = df_save_tag.merge(df_tags,how='inner',on='instance_id')
print(df_save_tag.shape)
df_save_notag = df_save_notag.merge(df_notags,how='inner',on='instance_id')
print(df_save_notag.shape)
df_save = train_ori[['instance_id']].copy()
df_save = df_save.merge(pd.concat([df_save_tag,df_save_notag],sort=False),how='inner',on='instance_id')
print(df_save.shape)

(1001650, 2)
(691880, 2)
(309770, 2)
(1001650, 2)


In [82]:
df_save.head(10)

,instance_id,predicted_score
0,6930856710792380886,0.139629
1,5460409694420131920,0.364073
2,982813438159141507,0.110336
3,529991959116679673,0.266944
4,5357053206615171780,0.005334
5,792454096205931185,0.168445
6,7020394095530516431,0.015871
7,3223889685220093136,0.255755
8,6022950513423196038,0.350819
9,8354622923398290680,0.272529


In [81]:
df_save.to_csv('tagsFrom2File.csv',index=False)

# TEST STD  -- holdout

In [63]:
from sklearn.metrics import log_loss,roc_auc_score
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_ori = pd.read_pickle(FILE.train_ori.value)

df_oof = pd.read_pickle('../train/new_model_rnnEBD.pkl')
predict_ho_all = df_oof.loc[holdout_index,'predicted_score'].values
y_ho_all = train_ori.loc[holdout_index,'click'].values

print('HOLDOUT ANALYSIS')
print('overall')
print(log_loss(y_ho_all,predict_ho_all))
print(roc_auc_score(y_ho_all,predict_ho_all))
print('---------------')

print('no tags')
holdout_notag = train_ori.loc[holdout_index].copy()
holdout_notag_index = holdout_notag[holdout_notag['user_tags'].isnull()].index
predict_ho_notag = df_oof.loc[holdout_notag_index,'predicted_score'].values
y_ho_notag = train_ori.loc[holdout_notag_index,'click'].values
print(log_loss(y_ho_notag,predict_ho_notag))
print(roc_auc_score(y_ho_notag,predict_ho_notag))
print('---------------')

print('has tag')
holdout_tag = train_ori.loc[holdout_index].copy()
holdout_tag_index = holdout_tag[holdout_tag['user_tags'].notnull()].index
predict_ho_tag = df_oof.loc[holdout_tag_index,'predicted_score'].values
y_ho_tag = train_ori.loc[holdout_tag_index,'click'].values
print(log_loss(y_ho_tag,predict_ho_tag))
print(roc_auc_score(y_ho_tag,predict_ho_tag))


print('\n')
print('OOF ALL ANALYSIS')
predict_all = df_oof['predicted_score'].values
y_all = train_ori['click'].values
print('overall')
print(log_loss(y_all,predict_all))
print(roc_auc_score(y_all,predict_all))
print('------------------------')

print('no tag')
notag = train_ori.copy()
notag_index = notag[notag['user_tags'].isnull()].index
predict_notag = df_oof.loc[notag_index,'predicted_score'].values
y_notag = train_ori.loc[notag_index,'click'].values
print(log_loss(y_notag,predict_notag))
print(roc_auc_score(y_notag,predict_notag))
print('---------------------')

print('has tag')
tag = train_ori.copy()
tag_index = tag[tag['user_tags'].notnull()].index
predict_tag = df_oof.loc[tag_index,'predicted_score'].values
y_tag = train_ori.loc[tag_index,'click'].values
print(log_loss(y_tag,predict_tag))
print(roc_auc_score(y_tag,predict_tag))

HOLDOUT ANALYSIS
overall
0.4245985252591007
0.7700736077067292
---------------
no tags
0.4461467055059244
0.7733677078367014
---------------
has tag
0.41280134646637623
0.766316717784432


OOF ALL ANALYSIS
overall
0.41522739757199445
0.7697014306209602
------------------------
no tag
0.4310607218865263
0.7614868345625095
---------------------
has tag
0.4081384675943792
0.7732163784288363


In [79]:
from sklearn.metrics import log_loss,roc_auc_score
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_ori = pd.read_pickle(FILE.train_ori.value)

# df_oof = pd.read_pickle('../train/new_model_RNN_OPTIMIZE_Utag_NA.pkl')
df_oof = df_save.copy()
predict_ho_all = df_oof.loc[holdout_index,'predicted_score'].values
y_ho_all = train_ori.loc[holdout_index,'click'].values

print('HOLDOUT ANALYSIS')
print('overall')
print(log_loss(y_ho_all,predict_ho_all))
print(roc_auc_score(y_ho_all,predict_ho_all))
print('---------------')

print('no tags')
holdout_notag = train_ori.loc[holdout_index].copy()
holdout_notag_index = holdout_notag[holdout_notag['user_tags'].isnull()].index
predict_ho_notag = df_oof.loc[holdout_notag_index,'predicted_score'].values
y_ho_notag = train_ori.loc[holdout_notag_index,'click'].values
print(log_loss(y_ho_notag,predict_ho_notag))
print(roc_auc_score(y_ho_notag,predict_ho_notag))
print('---------------')

print('has tag')
holdout_tag = train_ori.loc[holdout_index].copy()
holdout_tag_index = holdout_tag[holdout_tag['user_tags'].notnull()].index
predict_ho_tag = df_oof.loc[holdout_tag_index,'predicted_score'].values
y_ho_tag = train_ori.loc[holdout_tag_index,'click'].values
print(log_loss(y_ho_tag,predict_ho_tag))
print(roc_auc_score(y_ho_tag,predict_ho_tag))


print('\n')
print('OOF ALL ANALYSIS')
predict_all = df_oof['predicted_score'].values
y_all = train_ori['click'].values
print('overall')
print(log_loss(y_all,predict_all))
print(roc_auc_score(y_all,predict_all))
print('------------------------')

print('no tag')
notag = train_ori.copy()
notag_index = notag[notag['user_tags'].isnull()].index
predict_notag = df_oof.loc[notag_index,'predicted_score'].values
y_notag = train_ori.loc[notag_index,'click'].values
print(log_loss(y_notag,predict_notag))
print(roc_auc_score(y_notag,predict_notag))
print('---------------------')

print('has tag')
tag = train_ori.copy()
tag_index = tag[tag['user_tags'].notnull()].index
predict_tag = df_oof.loc[tag_index,'predicted_score'].values
y_tag = train_ori.loc[tag_index,'click'].values
print(log_loss(y_tag,predict_tag))
print(roc_auc_score(y_tag,predict_tag))

HOLDOUT ANALYSIS
overall
0.42427127315173174
0.7707807848108077
---------------
no tags
0.4453470427050435
0.7752816333339541
---------------
has tag
0.41273272935050637
0.7665308397063472


OOF ALL ANALYSIS
overall
0.41506035127171464
0.7699643234819564
------------------------
no tag
0.4306924221655436
0.7621449864728095
---------------------
has tag
0.40806152690797903
0.7733311925087457


In [89]:
from sklearn.metrics import log_loss,roc_auc_score
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_ori = pd.read_pickle(FILE.train_ori.value)

df_ooof = pd.read_pickle('../blend/trainoof_lgb_cv4153_ho4254_pb4249_tag4085_notag4307.pkl')
df_oof = train_ori[['instance_id']].copy()
df_oof = df_oof.merge(df_ooof,how='inner',on='instance_id')
print(df_oof.shape)
# df_oof = df_save.copy()
predict_ho_all = df_oof.loc[holdout_index,'predicted_score'].values
y_ho_all = train_ori.loc[holdout_index,'click'].values

print('HOLDOUT ANALYSIS')
print('overall')
print(log_loss(y_ho_all,predict_ho_all))
print(roc_auc_score(y_ho_all,predict_ho_all))
print('---------------')

print('no tags')
holdout_notag = train_ori.loc[holdout_index].copy()
holdout_notag_index = holdout_notag[holdout_notag['user_tags'].isnull()].index
predict_ho_notag = df_oof.loc[holdout_notag_index,'predicted_score'].values
y_ho_notag = train_ori.loc[holdout_notag_index,'click'].values
print(log_loss(y_ho_notag,predict_ho_notag))
print(roc_auc_score(y_ho_notag,predict_ho_notag))
print('---------------')

print('has tag')
holdout_tag = train_ori.loc[holdout_index].copy()
holdout_tag_index = holdout_tag[holdout_tag['user_tags'].notnull()].index
predict_ho_tag = df_oof.loc[holdout_tag_index,'predicted_score'].values
y_ho_tag = train_ori.loc[holdout_tag_index,'click'].values
print(log_loss(y_ho_tag,predict_ho_tag))
print(roc_auc_score(y_ho_tag,predict_ho_tag))


print('\n')
print('OOF ALL ANALYSIS')
predict_all = df_oof['predicted_score'].values
y_all = train_ori['click'].values
print('overall')
print(log_loss(y_all,predict_all))
print(roc_auc_score(y_all,predict_all))
print('------------------------')

print('no tag')
notag = train_ori.copy()
notag_index = notag[notag['user_tags'].isnull()].index
predict_notag = df_oof.loc[notag_index,'predicted_score'].values
y_notag = train_ori.loc[notag_index,'click'].values
print(log_loss(y_notag,predict_notag))
print(roc_auc_score(y_notag,predict_notag))
print('---------------------')

print('has tag')
tag = train_ori.copy()
tag_index = tag[tag['user_tags'].notnull()].index
predict_tag = df_oof.loc[tag_index,'predicted_score'].values
y_tag = train_ori.loc[tag_index,'click'].values
print(log_loss(y_tag,predict_tag))
print(roc_auc_score(y_tag,predict_tag))

(1001650, 2)
HOLDOUT ANALYSIS
overall
0.4254112004197366
0.7688069809737544
---------------
no tags
0.446491939958558
0.772482956997677
---------------
has tag
0.4138699356552102
0.7650510537943798


OOF ALL ANALYSIS
overall
0.41576059658816633
0.7686108537875833
------------------------
no tag
0.43142908847725814
0.7606146464832051
---------------------
has tag
0.4087454657381865
0.7720329301467128


In [ ]:
HOLDOUT ANALYSIS
overall
0.42427127315173174
0.7707807848108077
---------------
no tags
0.4453470427050435
0.7752816333339541
---------------
has tag
0.41273272935050637
0.7665308397063472


OOF ALL ANALYSIS
overall
0.41506035127171464
0.7699643234819564
------------------------
no tag
0.4306924221655436
0.7621449864728095
---------------------
has tag
0.40806152690797903
0.7733311925087457



(1001650, 2)
HOLDOUT ANALYSIS
overall
0.4254112004197366
0.7688069809737544
---------------
no tags
0.446491939958558
0.772482956997677
---------------
has tag
0.4138699356552102
0.7650510537943798


OOF ALL ANALYSIS
overall
0.41576059658816633
0.7686108537875833
------------------------
no tag
0.43142908847725814
0.7606146464832051
---------------------
has tag
0.4087454657381865
0.7720329301467128

In [ ]:
trainoof_lgb_cv4153_ho4254_pb4249_tag4085_notag4307.pkl

In [86]:
df_oof.shape

(1001650, 2)

In [87]:
np.sum(train_ori.instance_id.values != df_oof.instance_id.values)

999971